<a href="https://colab.research.google.com/github/tiagoassun/keyrus_teste_tecnico/blob/main/keyrus_teste_tecnico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste Técnico de Engenharia de Dados

## Baixando, instalando e configurando o Apache Spark

In [ ]:
# Instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixando Apche Spark que será salvo em no diretorio "/content/"
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [ ]:
# Baixando bibliotecas
!pip install -q findspark
!pip install pyspark[sql]
!pip install seaborn
!pip install geopandas

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=7dc9199490a70832564f242443b987ee9509f7aab5bb682ef8b06dd66b90241a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.3 MB 40.6 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 


In [ ]:
# Configurar as variáveis de ambiente do Linux que o Google Colab roda em cima
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [ ]:
# Tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

# Iniciar uma sessão local
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master('local[*]').getOrCreate()
sqlContext = SQLContext(spark)

# Setando cofiguração do Spark
spark.conf.set("spark.driver.maxResultSize", "0")
spark.conf.set("spark.executor.memory", "11g")
spark.conf.set("spark.driver.memory	", "11g")

## Criando conexão com o Google Drive e importando datasets

In [ ]:
# Criando conexão com o Google Drive da conta que está utilizando o Google Colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/
